In [1]:
import os
from os import path

books = []

for bookname in os.listdir('../data/dostoevsky'):
    book = open(path.join('../data/dostoevsky', bookname), 'r', encoding='utf-8').read().splitlines()
    book = [line for line in book if len(line) > 0]
    books.append(book)
    
data = [s for book in books for s in book]

In [2]:
with open('../data/generated/dostoevsky_joined.txt', 'w', encoding='utf-8') as f:
    for line in data:
        f.write(line + '\n')

In [3]:
%%bash

dostoevsky_joined="../data/generated/dostoevsky_joined.txt"
dostoevsky_sent_split="../data/generated/dostoevsky_sent_split.txt"
mosesdecoder="../ext-libs/mosesdecoder"

$mosesdecoder/scripts/ems/support/split-sentences.perl -l ru \
    < $dostoevsky_joined > $dostoevsky_sent_split

rm $dostoevsky_joined

Sentence Splitter v3
Language: ru


In [4]:
!wc -l ../data/generated/dostoevsky_sent_split.txt

100937 ../data/generated/dostoevsky_sent_split.txt


In [11]:
data = open('../data/generated/dostoevsky_sent_split.txt', 'r', encoding='utf-8').read().splitlines()

Let's tokenize our Dostoevsky dataset

In [13]:
%%bash

mosesdecoder="../ext-libs/mosesdecoder"
generated_data_dir="../data/generated"

threads=6

cat "$generated_data_dir/dostoevsky_sent_split.txt" | \
$mosesdecoder/scripts/tokenizer/normalize-punctuation.perl -l ru | \
$mosesdecoder/scripts/tokenizer/tokenizer.perl -threads $threads -l ru > \
$generated_data_dir/dostoevsky.tok

Tokenizer Version 1.1
Language: ru
Number of threads: 6


In [ ]:
import fasttext

num_threads = 6

model_trg = fasttext.skipgram('../data/generated/dostoevsky.tok',
                              '../trained_models/dostoevsky.tok.cbow',
                              dim=512, min_count=5, silent=0, thread=num_threads)